In [1]:
import torch

In [54]:
# function 1
#dy/dx
x = torch.tensor(3, requires_grad=True, dtype=torch.float32)
y = x**2

In [55]:
print(x)
print(y)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)


In [56]:
y.backward()

In [57]:
x.grad

tensor(6.)

In [58]:
# frunction 2
# dx/dz = dx/dy * dy/dz
x = torch.tensor(2, requires_grad=True, dtype=torch.float32)

y = x ** 2 # 4
z = 2*y + y**2 # 8 + 16 = 24
#z = torch.sin(y)

In [59]:
z.backward()

In [60]:
print("x", x)
print("y", y)
print("z", z)

x tensor(2., requires_grad=True)
y tensor(4., grad_fn=<PowBackward0>)
z tensor(24., grad_fn=<AddBackward0>)


In [61]:
# final derivative
x.grad

tensor(40.)

In [62]:
# cannot access y and z beacuse these are not leaf or root node in the computation graph.
# x is leaf node(staring node)
y.grad
z.grad

/var/folders/2s/t87m92j14g9b1yw35n324r5w0000gn/T/ipykernel_94350/191622969.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  y.grad
/var/folders/2s/t87m92j14g9b1yw35n324r5w0000gn/T/ipykernel_94350/191622969.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you acces

![Alt text]( single_nuron.png "Optional Title")


In [63]:
# dL/dw = dL/dy_pred * dy_pred/dz * dz/dw
# dL/db = dL/dy_pred * dy_pred/dz * dz/db

## Manual Grad Calculation

In [64]:
x = torch.tensor(6.7) #input
y = torch.tensor(0) #output

w = torch.tensor(1.0) # initial weight
b = torch.tensor(0.0) # initial bias

In [24]:
# binary cross-entropy loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [66]:
#forward pass
z = w * x + b

#prediction
y_pred = torch.sigmoid(z)

loss = binary_cross_entropy_loss(y_pred, y)

In [67]:
# manual gradient calcualtion for backprop
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [68]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw:.4f}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db:.4f}")

Manual Gradient of loss w.r.t weight (dw): 6.6918
Manual Gradient of loss w.r.t bias (db): 0.9988


# Calculation with AutoGrad

In [69]:
x = torch.tensor(6.7) #input
y = torch.tensor(0) #output

w = torch.tensor(1.0, requires_grad=True) # initial weight
b = torch.tensor(0.0, requires_grad=True) # initial bias

In [70]:
#forward pass
z = w * x + b

y_pred = torch.sigmoid(z)

loss = binary_cross_entropy_loss(y_pred, y)

In [71]:
print('z', z)
print('y_pred', y_pred)
print('loss', loss)

z tensor(6.7000, grad_fn=<AddBackward0>)
y_pred tensor(0.9988, grad_fn=<SigmoidBackward0>)
loss tensor(6.7012, grad_fn=<NegBackward0>)


In [72]:
#goal is to find dl/dw, so we start on the loss funciton to get the gradiant
loss.backward()

In [73]:
#get grads
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


# Gradient Cleaning

In [74]:
x = torch.tensor(2.0, requires_grad=True)

In [75]:
y = x ** 2
y.backward()

In [76]:
x.grad

tensor(4.)

In [ ]:
# reset grads to zero
x.grad.zero_() 

tensor(0.)

## Manual Training
Dataset:

| SSC  | HSC  | Chance |
|------|------|--------|
| 5.0  | 5.0  | 1      |
| 4.9  | 5.0  | 0      |
| 4.5  | 4.3  | 1      |



#### Training for 2 input features and one neuron

In [1]:
import torch

# Input and output tensors
x = torch.tensor([[5., 5.],
                  [4.9, 5.],
                  [4.5, 4.3]], dtype=torch.float32)
y = torch.tensor([1., 0., 1.], dtype=torch.float32)

# Weight and bias initialization
w = torch.tensor([[.3,.0]], requires_grad=True, dtype=torch.float32)
b = torch.tensor([1.], requires_grad=True, dtype=torch.float32)

In [4]:
# Forward pass: z = X * W + b
z = torch.matmul(x, w.T) + b

# Predictions with sigmoid activation
y_pred = torch.sigmoid(z)
y_pred = torch.squeeze(y_pred)  # Ensure shape is correct (1D tensor)

# Loss computation using BCELoss
loss_fn = torch.nn.BCELoss()
_loss = loss_fn(y_pred, y)

# Backpropagation
_loss.backward()


In [5]:
# Update weights and bias
lr = 0.001
with torch.no_grad():
    w -= lr * w.grad
    b -= lr * b.grad

# Reset gradients after update
w.grad.zero_()
b.grad.zero_()

print(f"Updated weights: {w}")
print(f"Updated bias: {b}")
print(f"Loss: {_loss.item()}")

Updated weights: tensor([[ 0.2988, -0.0013]], requires_grad=True)
Updated bias: tensor([0.9997], requires_grad=True)
Loss: 0.9070596098899841


In [ ]:
# train one single neuron for longer epochs

for i in range(500000):
    # Forward pass: z = X * W + b
    z = torch.matmul(x, w.T) + b

    # Predictions with sigmoid activation
    y_pred = torch.sigmoid(z)
    y_pred = torch.squeeze(y_pred)  # Ensure shape is correct (1D tensor)

    # Loss computation using BCELoss
    loss_fn = torch.nn.BCELoss()
    _loss = loss_fn(y_pred, y)
    print(f"Loss: {_loss.item()}")
    # Backpropagation
    _loss.backward()

    lr = 0.001
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad

    # Reset gradients after update
    w.grad.zero_()
    b.grad.zero_()

    print(f"Loss: {_loss.item()}")

In [8]:
print('w',w)
print('b',b)

w tensor([[ 7.1710, -7.7686]], requires_grad=True)
b tensor([3.5109], requires_grad=True)


In [9]:
# Make Predictions

# Trained weight and bias values
w1 = torch.tensor([[ 7.1710,-7.7686]], dtype=torch.float32)
b1 = torch.tensor([3.5109], dtype=torch.float32)

# Test input [5, 4.8] target[1]
x_test = torch.tensor([4.8, 4.8], dtype=torch.float32)

# Forward pass: z = x_test * w1 + b1
z_test = torch.matmul(x_test, w1.T) + b1

# Apply sigmoid activation to get the prediction
y_pred_test = torch.sigmoid(z_test)

print(f"Predicted output: {1 if y_pred_test.item() > .50 else 0}")

#Note: As we are using sigmoid, the predicted output is the pobability of belonging to the class labeled as 1. 

Predicted output: 1
